In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# URL: https://www.manythings.org/anki/ukr-eng.zip
# UKRAINIAN-ENGLISH
dataset = "ukr.txt"
batch_size = 64
epochs = 40
latent_dim = 256
num_samples = 10000

In [23]:
text_en = []
text_ukr = []
chars_en = set()
chars_ukr = set()
with open(dataset, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    text_en.append(input_text)
    text_ukr.append(target_text)
    for ch in input_text:
        if ch not in chars_en:
            chars_en.add(ch)
    for ch in target_text:
        if ch not in chars_ukr:
            chars_ukr.add(ch)

In [24]:
for _ in range(5):
    print(text_en[_])
    print(text_ukr[_])

Go.
	Йди.

Hi.
	Вітаю!

Hi.
	Привіт.

Hi.
	Привіт!

Run!
	Біжіть!



In [30]:
chars_en = sorted(list(chars_en))
chars_ukr = sorted(list(chars_ukr))
num_encoder_tokens = len(chars_en)
num_decoder_tokens = len(chars_ukr)
max_encoder_seq_length = max([len(txt) for txt in text_en])
max_decoder_seq_length = max([len(txt) for txt in text_ukr])

#input_token_index = dict([(ch, i) for i, ch in enumerate(chars_en)])
#target_token_index = dict([(ch, i) for i, ch in enumerate(chars_ukr)])
input_token_index = {ch: i for i, ch in enumerate(chars_en)}
target_token_index = {ch: i for i, ch in enumerate(chars_ukr)}

encoder_input_data = np.zeros((len(text_en), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(text_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(text_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
 
for i, (en, ukr) in enumerate(zip(text_en, text_ukr)):
    # Process English text
    for t, ch in enumerate(en):
        encoder_input_data[i, t, input_token_index[ch]] = 1.0
    # Fill remaining spaces with padding character
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    
    # Process Ukrainian text
    for t, ch in enumerate(ukr):
        decoder_input_data[i, t, target_token_index[ch]] = 1.0
        # Set target data for all characters except the first one
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[ch]] = 1.0
    # Fill remaining spaces with padding character
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [31]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [32]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("s2s")

Epoch 1/40
125/125 [==============================] - 23s 164ms/step - loss: 1.2943 - accuracy: 0.7391 - val_loss: 1.0067 - val_accuracy: 0.7270
Epoch 2/40
125/125 [==============================] - 20s 156ms/step - loss: 0.8450 - accuracy: 0.7750 - val_loss: 0.8529 - val_accuracy: 0.7740
Epoch 3/40
125/125 [==============================] - 19s 152ms/step - loss: 0.8392 - accuracy: 0.7884 - val_loss: 0.8168 - val_accuracy: 0.7777
Epoch 4/40
125/125 [==============================] - 19s 153ms/step - loss: 0.7027 - accuracy: 0.8080 - val_loss: 0.7396 - val_accuracy: 0.7952
Epoch 5/40
125/125 [==============================] - 19s 152ms/step - loss: 0.6475 - accuracy: 0.8182 - val_loss: 0.6990 - val_accuracy: 0.8010
Epoch 6/40
125/125 [==============================] - 19s 156ms/step - loss: 0.6151 - accuracy: 0.8243 - val_loss: 0.6696 - val_accuracy: 0.8087
Epoch 7/40
125/125 [==============================] - 20s 161ms/step - loss: 0.5913 - accuracy: 0.8301 - val_loss: 0.6493 - val_ac

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


In [33]:
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output 
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [34]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [38]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", text_en[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Go.
Decoded sentence: Запитайтеся.

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: Намай подала!

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: Намай подала!

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: Намай подала!

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Run!
Decoded sentence: Підійм!

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Run!
Decoded sentence: Підійм!

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Run!
Decoded sentence: Підійм!

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Wow!
Decoded sentence: Зачека!

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Wow!
Decoded sentence: Зачека!

1/1 [==============================] - 0s 25ms/step
-
Input sen